# Enoncé du problème

L'objectif de ce projet est d'estimer la longueur de câble sous-marin nécessaire pour relier deux côtes $A$ et $B$  en utilisant des simulations conditionnelles.


Le câble reposera sur le fond marin dont la profondeur est inconnue.
Le segment $[AB]$ est discrétisé par une séquence de (N+1) points. On pose $x_0=A$ et pour $i=1,\dots,N$, $$x_i=x_0+i\Delta$$ où $$\Delta = \frac{AB}{N}$$ de telle sorte que $x_N=B$.
On note $z(x)$ la profondeur du fond marin au point $x$ de telle sorte 
qu'on pourra estimer la longueur totale de câble nécessaire par la somme 
des longueurs sur les segments de la discrétisation :

$$l=\sum_{i=1}^N\sqrt{\Delta^2+(z(x_i)-z(x_{i-1}))^2}.$$

Enfin, notons que l'on dispose d'un ensemble de $n$ observations de la 
profondeur que l'on supposera situées sur des points de discrétisation $z(x_{j_1}),\dots,z(x_{j_n})$.


On adopte un modèle probabiliste pour la profondeur. On suppose que le vecteur des 
profondeurs sur les points de discrétisation 
$\mathbf{z}=(z(x_0),\dots,z(x_N))$ est la réalisation
d'un vecteur aléatoire gaussien $\mathbf{Z}=(Z(x_0),\dots,Z(x_N))$ 
dont le vecteur d'espérance ne contient qu'une seule valeur $\mu$ 
répétée $N+1$ fois et dont la matrice de covariance $\Sigma$ a pour termes $\sigma_{ij}$
définis par $\sigma_{ij}=C(|x_i-x_j|)$ où $C$ est une
fonction décroissante, traduisant le fait que deux points 
géographiquement proches ont tendance à avoir des profondeurs plus similaires que deux points éloignés.

On supposera que la matrice de covariance ainsi 
générée est définie-positive (en fait, $C$ sera choisie parmi les fonctions qui, 
appliquées aux termes d'une matrice de distance, produisent des matrices définie-positives). 

Si on note $L$ la variable aléatoire donnant la longueur de cable nécessaire : 
$$L=\sum_{i=1}^N\sqrt{\Delta^2+(Z(x_i)-Z(x_{i-1}))^2},$$
un bon estimateur de $L$ est fourni par l'espérance conditionnelle 

$$L^\star=E[L|Z(x_{j_1})=z(x_{j_1}),\dots,Z(x_{j_n})=z(x_{j_n})].$$
                                                                              
Cependant, cette quantité est difficilement accessible par le calcul. 
On va donc avoir recours à des
simulations conditionnelles. C'est-à-dire que l'on va simuler 
un nombre $K$ de réalités (disons des réalisations du modèle 
probabiliste choisi), et sur chacune d'entre elle, 
la quantité de câble nécessaire sera évaluée. 
On disposera ainsi d'un échantillon $l_{(1)},\dots,l_{(K)}$ de 
longueures simulées. Puis on approchera l'espérance conditionnelle  par 
$$L^\star=\frac{1}{K}\sum_{k=1}^K l_{(k)}.$$

L'objectif de ce projet est donc d'écrire un code permettant 
d'effectuer cette simulation conditionnelle, puis de l'appliquer 
au jeu de données fourni et d'en déduire une estimation de la longueur de câble nécessaire.

# Questions théoriques

1. Quel théorème du cours nous autorise-t-il à estimer l'espérance conditionnelle par la moyenne empirique de simulations conditionnelles ?

2. Rappeler la loi conditionnelle du vecteur des composantes de $\mathbf{Z}$ correspondant aux points de discrétisation
sans observation, connaissant les valeurs prises par les composantes aux sites d'observation.

3. Si $\mathbf{Y}=(Y_1,\dots,Y_p)$ est un vecteur de composantes gaussiennes indépendantes, toutes d'espérance nulle et de variance 1, 
quelle est la loi du vecteur $\mathbf{Z}=m+R\mathbf{Y}$ où $R$ est une matrice $p\times p$ inversible et $m$ est un vecteur de taille $p$ ?

4. En déduire un algorithme de simulation conditionnelle.

# Réponses questions théoriques

1. La suite de variables aléatoires des longueurs simulées $l_{(k)}$ est une suite de varaibles aléatoires $L^1$ indépendantes et de même loi donc d'après la loi des grands nombres la moyenne empirique de ces longeurs converge vers l'espérence de ces variables aléatoires qui est donc l'espérance conditionelle.

2. On a un vecteur gaussien $Z = (Z(x_0), . . . , Z(x_N))$ de vecteur d’espérance $\mu$ et de matrice de covariance C supposée définie positive. Alors la densité conditionnelle de Z (aux points de discrédisation sans observation) sachant $z = (z(x_{j_1}),..., z(x_{j_n}))$ s’écrit :

$f_{Z | z = (z(x_{j_1}),..., z(x_{j_n}))} (x) = \frac{1}{(2π)^\frac{N+1-n}{2} \sqrt{det(CS_Z)}} \exp(-\frac{1}{2}(x - \psi((z(x_{j_1}),..., z(x_{j_n}))))^T C S^{-1}(x - \psi((z(x_{j_1}),..., z(x_{j_n})))) $.

Où : $CS_Z = C_Z − C_{Z, z} C_{z}^{-1}C_{z, Z}$ lorsque l'on note $C_Z$ et $C_z$ les covariances des profondeurs aux points de non-observation et aux points d'observation. $C_{z, Z}$  est la covariance entre les profondeurs aux points de non-observation et les profondeurs aux points d'observation (c'est donc une matrice de taille $n \times (N+1-n)$) et on définit de manière symétrique $C_{Z, z}$ qui est donc sa transposée car $C$ est symétrique. Et $\psi(z) = m_Z + C_{Z, z} C_z^{-1} (z - m_z)$ où $m_Z$ et $m_z$ sont les espérances des profondeurs aux points de non-observation et d'observation (égaux à $\mu$ ici).


Donc la variable aléatoire $Z | z = (z(x_{j_1}),..., z(x_{j_n}))$ est gaussienne d’espérance $m_{Z | z = (z(x_{j_1}),..., z(x_{j_n})))} = \psi(z) = m_Z + C_{Z, z} C_z^{-1} (z - m_z)$ et de matrice de covariance $CS_Z = C_Z − C_{Z, z} C_{z}^{-1}C_{z, Z}$ .

3.  Si $Y = (Y_1, . . . , Y_p)$ est un vecteur de composantes gaussiennes indépendantes, toutes d’espérance nulle et de variance $C=1$ (c'est à dire l'identité), alors le vecteur $Z = m + RY$ (où $R$ est une matrice p × p et m est un vecteur de taille p) est gaussien d’espérance $\mathbb{E}(Z) = \mathbb{E}((m + RY)) = m + R \ \mathbb{E}(Y) = m$ et de variance  $\mathbb{V}(Z) =\mathbb{E}((Z-m)^2) = \mathbb{E}( RY(RY)^T) = R \ \mathbb{E}(Y(Y^T)) R^T = R C R^T = RR^T$ car ici $C = I_p$ et $Y$ est d'espérance nulle.

4. Pour simuler $Z | z = (z(x_{j_1}),..., z(x_{j_n}))$, on peut simuler un vecteur $Y = (Y_1, . . . , Y_{N+1-n})$ de composantes gaussiennes indépendantes, toutes d’espérance nulle et de variance $C=1$ (c'est à dire l'identité). On note $R$ la matrice de Cholesky de $CS_Z$. On a alors $Z | z = (z(x_{j_1}),..., z(x_{j_n})) = \mu + RY$. 

Pour simuler, Y il suffit de simuler $N+1-n$ vecteurs gaussiens d'espérance 0 et de variance 1 en utilisant la méthode du Box-Muller ou la fonction numpy.random.normal de Python.

# Données du problème
Conventionnellement, $A$ est l'origine, $B=500$, $N=100$.

Les données $$\begin{array}{c|r}i & z(x_i)\\
\hline
0 & 0\\
20 & -4\\
40 & -12.8\\
60 & -1\\
80 & -6.5\\
100 & 0\end{array}$$

L'espérance de chaque composante du vecteur aléatoire $\mathbf{Z}$ est donnée par $\mu=-5.$

La fonction $C$ est définie par $$C(h)=\sigma^2 e^{-|h|/a},$$

où $|h|$ correspond à la distance entre deux points, $a=50$ et $\sigma^2=12$.


# Implémentation

## Préambule

In [ ]:
#Chargement de dépendances

import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt

#Discrétisation
A=0
B=500
N=101 #Nombre de points de discrétisation
Delta = (B-A)/(N-1)
discretization_indexes = np.arange(N)
discretization = discretization_indexes*Delta
#Paramètres du modèle

mu=-5
a = 50
sigma2 = 12

#Données

observation_indexes = [0,20,40,60,80,100]
depth = np.array([0,-4,-12.8,-1,-6.5,0])

#Indices des composantes correspondant aux observations et aux componsantes non observées

unknown_indexes=list(set(discretization_indexes)-set(observation_indexes))


## Questions

1. Ecrire une fonction qui prend en argument la distance entre les points, le paramètre $a$, et le paramètre $\sigma^2$, et qui retourne la covariance entre deux points.
On pourra fournir une matrice de distance à cette fonction. Dans ce cas, la fonction renverra la matrice de covariance.

In [ ]:
def cov(dist, a = a, sigma2 = sigma2):
    def C(x) :
        return sigma2*np.exp(-np.abs(x)/a)
    return C(dist)

2. Calculer la matrice de distance.

In [ ]:
matrice_distance = np.array([[np.abs(i-j) for i in discretization] for j in discretization])

3. Calculer la matrice de covariance du vecteur $\mathbf{Z}=(Z(x_0),\dots,Z(x_N))$.

In [ ]:
matrice_covariance = cov(matrice_distance)

4. Extraire les 3 matrices de covariance suivantes :

 * entre les observations

 * entre les observations et les inconnues

 * entre les inconnues


In [ ]:
cov_obs = matrice_covariance[observation_indexes][:,observation_indexes]
cov_obs_unknown = matrice_covariance[observation_indexes][:,unknown_indexes]
cov_unknown = matrice_covariance[unknown_indexes][:,unknown_indexes]

5. Calculer l'espérance conditionnelle des composantes non observées connaissant les observations et la représenter avec les données.

In [ ]:
esp = np.array([mu]*N)
esp_cond = esp[unknown_indexes] + np.transpose(cov_obs_unknown).dot(np.linalg.inv(cov_obs)).dot(depth-esp[observation_indexes])

esp_concatene = []
j=0
for i in range(N):
    if i in observation_indexes:
        esp_concatene.append(depth[i//20])
    else :
        esp_concatene.append(esp_cond[j])
        j+=1
plt.plot(np.array(esp_concatene), label='Espérance conditionelle')
plt.plot(observation_indexes, np.array(depth), 'r', label = 'Données')
plt.legend()

In [ ]:
esp = np.array([mu]*N)
esp_cond = esp[unknown_indexes] + np.transpose(cov_obs_unknown).dot(np.linalg.inv(cov_obs)).dot(depth-esp[observation_indexes])

plt.plot(unknown_indexes, esp_cond, label='Espérance conditionelle')
plt.plot(observation_indexes, np.array(depth), 'r', label = 'Données')
plt.legend()

6. Calculer la matrice de covariance conditionnelle et tracer sa diagonale (variance conditionnelle) en fonction de la position. Commenter.

In [ ]:
cov_cond = cov_unknown - np.transpose(cov_obs_unknown).dot(np.linalg.inv(cov_obs)).dot(cov_obs_unknown)
diag = [cov_cond[i, i] for i in range(95)]
plt.plot(np.array(unknown_indexes)*Delta, diag)

C'est bizarre...

7. Effectuer une simulation conditionnelle. Sur un même graphique, tracer la simulation ainsi que les données et l'espérance conditionnelle. Commenter.

In [ ]:
Y = np.random.normal(0,1, 95)
R = np.linalg.cholesky(cov_cond)
Z_cond = esp_cond + R.dot(Y)
plt.plot(np.array(unknown_indexes)*Delta, Z_cond, label='Simulation conditionelle')
plt.plot(np.array(observation_indexes)*Delta, depth, 'r', label ='Données')
plt.plot(np.array(unknown_indexes)*Delta, esp_cond,'g', label = 'Espérance conditionelle')
plt.legend()

C'est encore étrange

8. Ecrire une fonction qui calcule la longueur du câble en fonction du vecteur des profondeurs et du pas de discrétisation.

In [ ]:
def long_cable(Z, delta= Delta):
    l = 0
    for i in range(1, N):
        l+= np.sqrt(delta**2+ (Z[i]-Z[i-1])**2)
    return l       

9. Utiliser cette fonction pour calculer la longueur du câble à partir de 100 simulations. Comparer l'espérance conditionnelle (estimée) de la longueur avec la longueur de l'espérance conditionnelle.

In [ ]:
def concatenation(Z):
    profondeur = []
    j=0
    for i in range(N):
        if i in observation_indexes:
            profondeur.append(depth[i//20])
        else :
            profondeur.append(Z[j])
            j+=1
    return profondeur

In [ ]:
long_est = []
#R = np.linalg.cholesky(cov_cond) #à revoir : pourquoi matrice_covariance et pas cov_cond ?

for i in range(100):
    Y = np.random.normal(0, 1, 95)
    Z = esp_cond + R.dot(Y)
    profondeur = concatenation(Z)
    long_est.append(long_cable(profondeur))
    plt.plot(np.array(unknown_indexes)*Delta, Z, 'b')

plt.plot(np.array(observation_indexes)*Delta, depth, 'r', label ='Données')
plt.plot(np.array(unknown_indexes)*Delta, esp_cond,'g', label = 'Espérance conditionelle')
plt.legend()
l = sum(long_est)/len(long_est)
long_esp = long_cable(concatenation(esp_cond))
print(f'La longueur moyenne simulée est {round(l,2)} m et celle calculée à partir de l\'espérance conditionelle est {round(long_esp,2)} m.')

10. Représenter la suite $M_n$ des moyennes des longueurs de câbles en fonction du nombre de simulations. Commenter.

In [ ]:
def M(n):
    long_est = []
    for i in range(n):
        Y = np.random.normal(0, 1, 95)
        Z = esp_cond + R.dot(Y)
        profondeur = concatenation(Z)
        long_est.append(long_cable(profondeur))
    return(sum(long_est)/len(long_est))

nb_simu = [1*i for i in range(1,100)]
moy_cable = [M(i) for i in nb_simu]
plt.plot(nb_simu, moy_cable)


11. Représenter l'histogramme des longueurs de câbles générées.

In [ ]:
plt.hist(long_est, 20)[2]

12. Donner un intervalle de confiance à 95% de la longueur du câble par 2 méthodes différentes. Commenter.

__Première méthode__ : On regarde sur l'histogramme précédent l'intervalle centré sur la valeur moyenne dans lequel se situent 95 (96 ici) des 100 valeurs.

In [ ]:
long_est.sort()
indice_moyenne = 0
while long_est[indice_moyenne]<l :
    indice_moyenne +=1
print(f'Il y a 96% de chance pour que la véritable longueur se situe dans l\'intervalle [{round(long_est[max(indice_moyenne -48,0)],2)} m, {round(long_est[min(indice_moyenne +48, 99)],2)} m].')

__Deuxième méthode__ : On utilise la formule de l'intervalle de confiance à 95%: $[l - \frac{1}{\sqrt{n}}; l - \frac{1}{\sqrt{n}}]$.

In [ ]:
print(f'Il y a 96% de chance pour que la véritable longueur se situe dans l\'intervalle [{round(l-1/np.sqrt(100),2)} m, {round(l+1/np.sqrt(100),2)} m].')

13. Donner une estimation de la probabilité que la longueur du câble dépasse 525 m.

In [ ]:
indice = 0
while long_est[indice]<525 :
    indice +=1
print(f'Il y a {indice/100}% de chance pour que la longueur du cable soit supérieure à 525m.')

14. Reprendre les questions précédentes avec 1000, 10000 puis 100000 simulations. Commenter.

**Pour 1000 :**

In [ ]:
long_est = []
for i in range(1000):
    Y = np.random.normal(0, 1, 95)
    Z = esp_cond + R.dot(Y)
    profondeur = concatenation(Z)
    long_est.append(long_cable(profondeur))
l = sum(long_est)/len(long_est)
print(f'La longueur moyenne simulée est {round(l,2)} m et celle calculée à partir de l\'espérance conditionelle est {round(long_esp,2)} m.')

long_est.sort()
indice_moyenne = 0
while long_est[indice_moyenne]<l :
    indice_moyenne +=1
print(f'Il y a 95% de chance pour que la véritable longueur se situe dans l\'intervalle [{round(long_est[max(indice_moyenne -475,0)],2)} m, {round(long_est[min(indice_moyenne +475, 999)],2)} m].')

indice = 0
while long_est[indice]<525 :
    indice +=1
print(f'Il y a {indice/1000}% de chance pour que la longueur du cable soit supérieure à 525m.')

plt.hist(long_est, 100)[2]

**Pour 10 000 :**

In [ ]:
long_est = []
for i in range(10000):
    Y = np.random.normal(0, 1, 95)
    Z = esp_cond + R.dot(Y)
    profondeur = concatenation(Z)
    long_est.append(long_cable(profondeur))
l = sum(long_est)/len(long_est)
print(f'La longueur moyenne simulée est {round(l,2)} m et celle calculée à partir de l\'espérance conditionelle est {round(long_esp,2)} m.')

long_est.sort()
indice_moyenne = 0
while long_est[indice_moyenne]<l :
    indice_moyenne +=1
print(f'Il y a 95% de chance pour que la véritable longueur se situe dans l\'intervalle [{round(long_est[max(indice_moyenne -95*100//2,0)],2)} m, {round(long_est[min(indice_moyenne +95*100//2, 9999)],2)} m].')

indice = 0
while long_est[indice]<525 :
    indice +=1
print(f'Il y a {indice/10000}% de chance pour que la longueur du cable soit supérieure à 525m.')

plt.hist(long_est, 100)[2]

**Pour 100 000 :**

In [ ]:
long_est = []
for i in range(100000):
    Y = np.random.normal(0, 1, 95)
    Z = esp_cond + R.dot(Y)
    profondeur = concatenation(Z)
    long_est.append(long_cable(profondeur))
l = sum(long_est)/len(long_est)
print(f'La longueur moyenne simulée est {round(l,2)} m et celle calculée à partir de l\'espérance conditionelle est {round(long_esp,2)} m.')

long_est.sort()
indice_moyenne = 0
while long_est[indice_moyenne]<l :
    indice_moyenne +=1
print(f'Il y a 95% de chance pour que la véritable longueur se situe dans l\'intervalle [{round(long_est[max(indice_moyenne -95*1000//2,0)],2)} m, {round(long_est[min(indice_moyenne +95*1000//2, 99999)],2)} m].')

indice = 0
while long_est[indice]<525 :
    indice +=1
print(f'Il y a {indice/100000}% de chance pour que la longueur du cable soit supérieure à 525m.')

plt.hist(long_est, 1000)[2]